# ChatBot

Para el siguiente ejercicio, utilizaremos un ChatBot con definido por reglas. (Ejemplo tomado de: https://www.analyticsvidhya.com/blog/2021/07/build-a-simple-chatbot-using-python-and-nltk/)

In [ ]:
!pip install nltk

In [ ]:
import nltk
from nltk.chat.util import Chat, reflections

**Chat**: es la clase que contiene toda lo logica para processar el texto que el chatbot recibe y encontrar informacion util.   
**reflections**: es un diccionario que contiene entradas basica y sus correspondientes salidas.

In [ ]:
print(reflections)

Comenzemos contruyendo las reglas. Las siguientes lineas generan un conjunto de reglas simples.

In [ ]:
pairs = [
    [
        r"my name is (.*)",
        ["Hello %1, How are you today ?",]
    ],
    [
        r"hi|hey|hello",
        ["Hello", "Hey there",]
    ], 
    [
        r"what is your name ?",
        ["I am a bot created by Analytics Vidhya. you can call me crazy!",]
    ],
    [
        r"how are you ?",
        ["I'm doing goodnHow about You ?",]
    ],
    [
        r"sorry (.*)",
        ["Its alright","Its OK, never mind",]
    ],
    [
        r"I am fine",
        ["Great to hear that, How can I help you?",]
    ],
    [
        r"i'm (.*) doing good",
        ["Nice to hear that","How can I help you?:)",]
    ],
    [
        r"(.*) age?",
        ["I'm a computer program dudenSeriously you are asking me this?",]
    ],
    [
        r"what (.*) want ?",
        ["Make me an offer I can't refuse",]
    ],
    [
        r"(.*) created ?",
        ["Raghav created me using Python's NLTK library ","top secret ;)",]
    ],
    [
        r"(.*) (location|city) ?",
        ['Indore, Madhya Pradesh',]
    ],
    [
        r"how is weather in (.*)?",
        ["Weather in %1 is awesome like always","Too hot man here in %1","Too cold man here in %1","Never even heard about %1"]
    ],
    [
        r"i work in (.*)?",
        ["%1 is an Amazing company, I have heard about it. But they are in huge loss these days.",]
    ],
    [
        r"(.*)raining in (.*)",
        ["No rain since last week here in %2","Damn its raining too much here in %2"]
    ],
    [
        r"how (.*) health(.*)",
        ["I'm a computer program, so I'm always healthy ",]
    ],
    [
        r"(.*) (sports|game) ?",
        ["I'm a very big fan of Football",]
    ],
    [
        r"who (.*) sportsperson ?",
        ["Messy","Ronaldo","Roony"]
    ],
    [
        r"who (.*) (moviestar|actor)?",
        ["Brad Pitt"]
    ],
    [
        r"i am looking for online guides and courses to learn data science, can you suggest?",
        ["Crazy_Tech has many great articles with each step explanation along with code, you can explore"]
    ],
    [
        r"quit",
        ["BBye take care. See you soon :) ","It was nice talking to you. See you soon :)"]
    ],
]

Después de definir las reglas, definimos la función para ejecutar el proceso de chat.

In [ ]:
def chat(this_creator='Nelson Yalta'):
    print(f"Hola!!! Yo soy un chatbot creado por {this_creator}, y estoy listo para sus servicios. Recuede que hablo inglés.")
    chat = Chat(pairs, reflections)
    chat.converse()

chat()

## ChatBot Parlanchin

Vamos a llamar nuestros modelos preentrandos para reconocimiento y sintetizacion de voz

### ASR

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q espnet_model_zoo
!pip install git+git://github.com/ricardodeazambuja/colab_utils.git
!pip install -q pyopenjtalk==0.1.5 parallel_wavegan==0.5.3 

Cargamos nuestras librerias

In [ ]:
from colab_utils import getAudio
import numpy as np
from matplotlib import pyplot as plt
from librosa import resample
import time
import torch
import string
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text

Definimos la configuracion

In [ ]:
asr_lang = 'en'
fs = 16000
asr_tag = 'Shinji Watanabe/spgispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_unnorm_bpe5000_valid.acc.ave'

In [ ]:
d = ModelDownloader("/content/drive/MyDrive/Data_NB/pretrained")
# It may takes a while to download and build models
speech2text = Speech2Text(
    **d.download_and_unpack(asr_tag),
    device="cuda",
    minlenratio=0.0,
    maxlenratio=0.0,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

Grabamos nuestra voz

In [ ]:
speech, fs = getAudio()
dtype = speech.dtype
speech = speech.astype(np.float32) / (np.iinfo(dtype).max + 1)
speech = resample(speech, fs, 16000)
plt.plot(speech)

Probamos que funcione:

In [ ]:
hyp = 'Today I will share information about deep learning'
nbests = speech2text(speech)

text, *_ = nbests[0]
print(f"ASR hypothesis: {text_normalizer(text)}")

### TTS

Definimos nuestros parametros para TTS

In [ ]:
tts_lang = 'English'
tts_tag = 'kan-bayashi/ljspeech_vits' #@param ["kan-bayashi/ljspeech_tacotron2", "kan-bayashi/ljspeech_fastspeech", "kan-bayashi/ljspeech_fastspeech2", "kan-bayashi/ljspeech_conformer_fastspeech2", "kan-bayashi/ljspeech_vits"] {type:"string"}
vocoder_tag = "none" #@param ["none", "parallel_wavegan/ljspeech_parallel_wavegan.v1", "parallel_wavegan/ljspeech_full_band_melgan.v2", "parallel_wavegan/ljspeech_multi_band_melgan.v2", "parallel_wavegan/ljspeech_hifigan.v1", "parallel_wavegan/ljspeech_style_melgan.v1"] {type:"string"}


Cargamos las librerias

In [ ]:
from espnet2.bin.tts_inference import Text2Speech
from espnet2.utils.types import str_or_none

In [ ]:
text2speech = Text2Speech.from_pretrained(
    model_tag=str_or_none(tag),
    vocoder_tag=str_or_none(vocoder_tag),
    device="cuda",
    # Only for Tacotron 2 & Transformer
    threshold=0.5,
    # Only for Tacotron 2
    minlenratio=0.0,
    maxlenratio=10.0,
    use_att_constraint=False,
    backward_window=1,
    forward_window=3,
    # Only for FastSpeech & FastSpeech2 & VITS
    speed_control_alpha=1.0,
    # Only for VITS
    noise_scale=0.667,
    noise_scale_dur=0.8,
)

Ahora, probamos el modelo

In [ ]:
# decide the input sentence by yourself
print(f"Input your favorite sentence in {lang}.")
x = input()

# synthesis
with torch.no_grad():
    start = time.time()
    wav = text2speech(x)["wav"]
rtf = (time.time() - start) / (len(wav) / text2speech.fs)
print(f"RTF = {rtf:5f}")

# let us listen to generated samples
from IPython.display import display, Audio
display(Audio(wav.view(-1).cpu().numpy(), rate=text2speech.fs))

### A mezclar todo:

Ahora, modificamos un poco para poder manejar la entada y salida

In [ ]:
class CustomChat(Chat):
    # Hold a conversation with a chatbot
    def converse(self, quit="quit"):
        user_input = ""
        while user_input != quit:
            user_input = quit
            try:
                user_input = input(">")
            except EOFError:
                print(user_input)
            if user_input:
                while user_input[-1] in "!.":
                    user_input = user_input[:-1]
                print(self.respond(user_input))